# Anthropic function calling with Langchain Agents

In this notebook we leveraage Langchain agents to perform function calling tasks. 

First we install the necessary python packages such as `duckduckgo-search`, `wikipedia`, and `xmltodict`, amongst others. 

- Initialize connection to **anthropic.claude-v2** using Bedrock
- Define and test the following tools individually
  - Wikipedia
  - Web Search and
  - PubMed search
- Finally, we put this all together by initializing a Langchain agent executor

In [1]:
!pip install duckduckgo-search wikipedia unstructured xmltodict --quiet

In [2]:
%load_ext rich

In [3]:
import json
import warnings

import boto3
from anthropic_bedrock import AI_PROMPT, HUMAN_PROMPT, AnthropicBedrock
from IPython.display import Markdown
from langchain.agents import AgentType, Tool, initialize_agent, load_tools
from langchain.llms.bedrock import Bedrock
from langchain.tools import DuckDuckGoSearchRun, PubmedQueryRun, tool
from langchain.utilities import WikipediaAPIWrapper
from rich import print

warnings.filterwarnings("ignore")

In [4]:
region = "us-west-2"
bedrock_runtime = boto3.client(
    service_name="bedrock-runtime", region_name=region
)  # bedrock runtime boto3 client
bedrock_model_id = "anthropic.claude-v2"  # Amazon Bedrock Model ID for claude-v2

# anthropic claude model parameters
model_kwargs = {
    "max_tokens_to_sample": 4096,
    "temperature": 0.0,
    "top_p": 0.999,
    "top_k": 250,
    "stop_sequences": [HUMAN_PROMPT],
}

# Define the llm with langchain.llms.bedrock
bedrock_llm = Bedrock(
    model_id=bedrock_model_id, client=bedrock_runtime, model_kwargs=model_kwargs
)

## Let's define a few tools

- [Wikipedia](https://wikipedia.org/) (Search wikipedia pages)
- Web Search (Search web using [DuckDuckGo](https://duckduckgo.com/) search engine)
- [PubMed](https://pubmed.ncbi.nlm.nih.gov/) (Medical papers and articles)

#### 01.Wikipedia Tool

[Wikipedia](https://wikipedia.org/) is a multilingual free online encyclopedia written and maintained by a community of volunteers, through open collaboration. Wikipedia is the largest and most-read reference work in history. 

>*NOTE:* To use this tool, install `wikipedia` package.

In [5]:
# Define the API wrapper for Wikipedia
wikipedia = WikipediaAPIWrapper()

# Define the Wikipedia tool with a description and the function to retrieve results from Wikipedia
wikipedia_tool = Tool(
    name="wikipedia",
    func=wikipedia.run,
    description="Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.",
)

# Test the defined Wikipedia tool
# Markdown(wikipedia_tool("What is the Archimedes Principle?"))

---

#### 02.DuckDuckGo Search Tool

[DuckDuckGo](https://duckduckgo.com/) is an internet privacy company most popularly known for their private search engine. The company emphasizes privacy and anonimity as one of the key principles behind all their products.

>*NOTE:* To use this tool, install `duckduckgo-search` package.

In [6]:
# Define the API wrapper for DuckDuckGo search
duckduckgo_search = DuckDuckGoSearchRun()

# Define the DuckDuckGo tool using a description and the function to retrieve results from DuckDuckGo search
duckduckgo_tool = Tool(
    name="DuckDuckGoSearch",
    func=duckduckgo_search.run,
    description="useful for when you need to answer questions about current events",
)

# Test the DuckDuckGo tool
# Markdown(duckduckgo_tool("Weather in Seattle, Washington?"))

---

#### 03.PubMed Tool

[PubMed](https://pubmed.ncbi.nlm.nih.gov/) is a free search engine for medical papers and articles.

>*NOTE:* To use this tool, install `xmltodict` package.

In [7]:
# Define the API wrapper for PubMed search
pubmed_search = PubmedQueryRun()

# Define the PubMed tool using a description and the function to retrieve results from PubMed
pubmed_tool = Tool(
    name="PubmedQueryRun",
    func=pubmed_search.run,
    description="Useful for when you need medical information",
)

# Test PubMed tool
# Markdown(pubmed_tool("Covid diagnosis"))

## Putting it all together

- Define all the tools as a list
- Initialize the agent with tool list and `AgentType` of type `ZERO_SHOT_REACT_DESCRIPTION`
- Explicity tell the agent to handle parsing errors

In [8]:
# Define a list of tools for the agent
tools = [wikipedia_tool, duckduckgo_tool, pubmed_tool]

# Initialize the agent with access to all the tooks in the list
agent_executor = initialize_agent(
    tools,
    bedrock_llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True,
)

output_dict = agent_executor.invoke(
    {"input": "Tell me about SageMaker serverless inference endpoints"}
)
print(output_dict)



> Entering new AgentExecutor chain...
 Here is my response:

Thought: To answer a question about SageMaker serverless inference endpoints, Wikipedia likely does not have detailed technical information. DuckDuckGo may have some high-level overviews but not in-depth details. PubMed is not relevant. The best action is to explain what I know about SageMaker serverless inference endpoints.

Final Answer: SageMaker serverless inference endpoints allow you to deploy machine learning models for real-time inference without having to manage servers. They automatically scale up and down to meet traffic demands with no idle capacity. You only pay for the compute time used for inferences. They are a fully managed, elastic inference service. Some key benefits are no server management, automatic scaling, sub-second latency, and pay-per-use pricing. They are useful for deploying production ML models and powering real-time prediction services.

> Finished chain.


{
    'input': 'Tell me about SageMaker serverless inference endpoints',
    'output': 'SageMaker serverless inference endpoints allow you to deploy machine learning models for real-time 
inference without having to manage servers. They automatically scale up and down to meet traffic demands with no 
idle capacity. You only pay for the compute time used for inferences. They are a fully managed, elastic inference 
service. Some key benefits are no server management, automatic scaling, sub-second latency, and pay-per-use 
pricing. They are useful for deploying production ML models and powering real-time prediction services.'
}